In [1]:
import os
import tqdm
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import darts.models as models
import logging
from darts import TimeSeries

In [2]:
# 设置绘图风格
plt.rcParams["font.sans-serif"] = ["SimHei"]
matplotlib.rcParams["axes.unicode_minus"] = False

In [3]:
# 编码格式为gb18030
day_data = pd.read_table("./HSI_日线数据.txt", encoding="gb18030", skiprows=1)
day_data = day_data.iloc[:-1]
day_data.columns = day_data.columns.str.strip()

In [4]:
series = TimeSeries.from_dataframe(day_data, value_cols=["收盘"], time_col="日期", freq="D")
series_df = series.pd_dataframe()
series_df = series_df.interpolate(method="linear")
series = TimeSeries.from_dataframe(series_df)

In [5]:
week_series = series.resample("W").mean()
month_series = series.resample("M").mean()

In [6]:
def train_predict_model(series, start_day, end_day, model):
    train_range = pd.date_range(start=start_day, end=end_day, freq="D")
    train = series.slice(train_range[0], train_range[-1])
    model.fit(train)
    return model


def train_predict_model_week(series, start_day, end_day, model):
    train_range = pd.date_range(start=start_day, end=end_day, freq="W")
    train = series.slice(train_range[0], train_range[-1])
    model.fit(train)
    return model


def train_predict_model_month(series, start_day, end_day, model):
    train_range = pd.date_range(start=start_day, end=end_day, freq="M")
    train = series.slice(train_range[0], train_range[-1])
    model.fit(train)
    return model

In [9]:
def cal_14_days_expected_return(
    series,
    duration_start="2020-11-17",
    duration_end="2023-11-17",
    moeel_train_start="2016-07-12",
    model="Prophet",
    csv_path="./Prophet_rate_of_return.csv",
):
    daily_rate_of_return = []

    for day in tqdm.tqdm_notebook(
        pd.date_range(start=duration_start, end=duration_end, freq="D"),
        desc="预测",
    ):
        if model == "ExponentialSmoothing":
            model = models.ExponentialSmoothing()

        if model == "Prophet":
            model = models.Prophet()

        if model == "StatsForecastAutoARIMA":
            model = models.StatsForecastAutoARIMA()

        if model == "TCN":
            model = models.TCNModel(
                input_chunk_length=21,
                output_chunk_length=14,
                n_epochs=10,
                batch_size=1024,
            )

        if model == "Dlinear":
            model = models.DLinearModel(
                input_chunk_length=21,
                output_chunk_length=14,
                n_epochs=10,
                batch_size=1024,
            )

        if model == "TiDE":
            model = models.TiDEModel(
                input_chunk_length=21,
                output_chunk_length=3,
                n_epochs=20,
                batch_size=1024,
            )

        model = train_predict_model(series, moeel_train_start, day, model)
        # 预测未来14天的股价
        pred = model.predict(14)
        predict_first_day_data = pred.pd_dataframe().iloc[0]
        predict_last_day_data = pred.pd_dataframe().iloc[-1]
        # 预测出来的十四天趋势的收益
        rate_of_return = (
            predict_last_day_data["收盘"] - predict_first_day_data["收盘"]
        ) / predict_first_day_data["收盘"]

        daily_rate_of_return.append(rate_of_return)

    daily_rate_of_return_df = pd.DataFrame(daily_rate_of_return, columns=["收益率"])
    dates = pd.date_range(duration_start, duration_end, freq="D")
    daily_rate_of_return_df.set_index(dates, inplace=True)
    daily_rate_of_return_df.to_csv(csv_path)

    return daily_rate_of_return_df

In [21]:
# use ExponentialSmoothing model
ExponentialSmoothing_daily_rate_of_return_df = cal_14_days_expected_return(
    series=series,
    model="ExponentialSmoothing",
    csv_path="./ExponentialSmoothing_rate_of_return.csv",
    duration_start="2020-11-17",
    duration_end="2023-11-17",
)

logging.disable(logging.CRITICAL)
# use Prophet model
Prophet_daily_rate_of_return_df = cal_14_days_expected_return(
    series=series,
    model="Prophet",
    csv_path="./prophet_rate_of_return.csv",
    duration_start="2020-11-17",
    duration_end="2023-11-17",
)

# use StatsForecastAutoARIMA model
ARIMA_daily_rate_of_return_df = cal_14_days_expected_return(
    series=series,
    model="StatsForecastAutoARIMA",
    csv_path="./StatsForecastAutoARIMA_rate_of_return.csv",
    duration_start="2020-11-17",
    duration_end="2023-11-17",
)

# use TCN model
TCN_daily_rate_of_return_df = cal_14_days_expected_return(
    series=series,
    model="TCN",
    csv_path="./TCN_rate_of_return.csv",
    duration_start="2020-11-17",
    duration_end="2023-11-17",
)

# use Dlinear model
Dlinear_daily_rate_of_return_df = cal_14_days_expected_return(
    series=series,
    model="Dlinear",
    csv_path="./Dlinear_rate_of_return.csv",
    duration_start="2020-11-17",
    duration_end="2023-11-17",
)

# use TiDE model
TiDE_daily_rate_of_return_df = cal_14_days_expected_return(
    series,
    duration_start="2020-11-17",
    duration_end="2023-11-17",
    moeel_train_start="2016-07-12",
    model="TiDE",
    csv_path="./TiDE_rate_of_return.csv",
)

预测:   0%|          | 0/1096 [00:00<?, ?it/s]

Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name            | Type             | Params
-----------------------------------------------------
0 | criterion       | MSELoss          | 0     
1 | train_metrics   | MetricCollection | 0     
2 | val_metrics     | MetricCollection | 0     
3 | decomposition   | _SeriesDecomp    | 0     
4 | linear_seasonal | Linear           | 308   
5 | linear_trend    | Linear           | 308   
-----------------------------------------------------
616       Trainable params
0         Non-trainable params
616       Total p

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name            | Type             | Params
-----------------------------------------------------
0 | criterion       | MSELoss          | 0     
1 | train_metrics   | MetricCollection | 0     
2 | val_metrics     | MetricCollection | 0     
3 | decomposition   | _SeriesDecomp    | 0     
4 | linear_seasonal | Linear           | 308   
5 | linear_trend    | Linear           | 308   
-----------------------------------------------------
616       Trainable params
0         Non-trainable params
616       Total p

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: |          | 0/? [00:00<?, ?it/s]